In [1]:
from localization_scripts.imports import *

In [ ]:
filename = '/home/smlm-workstation/event-smlm/our_ev_smlm_recordings/MT_5May_S2_reduced_bias_580sec/MT_5May_S2_reduced_bias_580sec.raw'
if os.path.basename(filename)[-4:] == ".raw":
    events = raw_events_to_array(filename).astype(
        [("x", "uint16"), ("y", "uint16"), ("p", "byte"), ("t", "uint64")]
    )

slice = events[(events["t"] < 50e6)]